## Implementation Checklist & Next Steps

### Phase 1: Dependencies & Setup
- [ ] Install `@floating-ui/react` and `recharts`: `npm install @floating-ui/react recharts`
- [ ] Update imports in MistakeNotebookPage.jsx
- [ ] Add new helper functions (calculateMasteryPriority, getMasteryState, etc.)

### Phase 2: Core Features
- [ ] Replace SmartTooltip with TooltipWithPortal (Floating UI)
- [ ] Implement ISRS priority scoring
- [ ] Add Rule of 3 archiving logic
- [ ] Create CalendarHeatmap component
- [ ] Create TopicHeatmap and ImprovementTrendChart components

### Phase 3: UI Integration
- [ ] Add "AI Daily Mission" button (uses selectAIDailyMission)
- [ ] Add Error Type tagging dropdown for each mistake
- [ ] Add ErrorTagSelector component to mistake cards
- [ ] Update filter UI for nested topic/subtopic selection
- [ ] Integrate CalendarHeatmap in main render
- [ ] Integrate Learning Analytics Dashboard

### Phase 4: State Management & Persistence
- [ ] Add state variables for errorTags, archivedMistakes, recentQuizTopics
- [ ] Implement useEffect hooks for localStorage sync
- [ ] Implement useMemo for performance optimization
- [ ] Test Rule of 3 archiving workflow

### Phase 5: Testing & Refinement
- [ ] Test ISRS score calculation with various attempt counts
- [ ] Verify tooltip edge detection and flipping
- [ ] Test interleaved practice selection (≥3 topics)
- [ ] Verify localStorage persistence across sessions
- [ ] Test i18n integration for all new text

### Key i18n Keys to Add
```javascript
{
  notebook: {
    aiDailyMission: "AI Daily Mission",
    mistakeClearingActivity: "30-Day Mistake-Clearing Activity",
    errorDensityByTopic: "Error Density by Topic",
    improvementTrend: "Improvement Trend",
    tagError: "Tag Error Type",
    less: "Less",
    more: "More"
  }
}
```

### Performance Considerations
- **useMemo**: Wrap ISRS priority calculations (depends on: mistakes, recentQuizTopics)
- **useMemo**: Wrap error density map (depends on: mistakes)
- **useMemo**: Wrap activity heatmap data (depends on: improvements)
- **Debounce**: Filter changes to prevent excessive re-calculations
- **Lazy Load**: Consider lazy-loading chart components if performance degrades

### localStorage Schema
```javascript
{
  "mistake_improvements": { // existing
    "question_id": { correctCount, lastCorrect, ... }
  },
  "mistake_archive": { // new
    "question_id": { correctCount, lastCorrect, archivedAt, ... }
  },
  "mistake_error_tags": { // new
    "question_id": "calculation" // error type
  },
  "recent_quiz_topics": [ // new
    "Topic1", "Topic2", "Topic3"
  ]
}
```

In [ ]:
// ═══════════════════════════════════════════════════════════════════════════════
// 8. MAIN COMPONENT STATE VARIABLES & HOOKS
// ═══════════════════════════════════════════════════════════════════════════════

/**
 * Key state variables to add to MistakeNotebookPage component:
 */

// Existing:
// const [mistakes, setMistakes] = useState([]);
// const [improvements, setImprovements] = useState({});
// const [selectedTopics, setSelectedTopics] = useState([]);
// const [selectedSubtopics, setSelectedSubtopics] = useState([]);

// NEW:
const [errorTags, setErrorTags] = useState(() => 
  JSON.parse(localStorage.getItem('mistake_error_tags') || '{}')
);

const [archivedMistakes, setArchivedMistakes] = useState(() =>
  JSON.parse(localStorage.getItem('mistake_archive') || '{}')
);

const [recentQuizTopics, setRecentQuizTopics] = useState(() =>
  JSON.parse(localStorage.getItem('recent_quiz_topics') || '[]')
);

// ───────────────────────────────────────────────────────────────────────────────

/**
 * KEY EFFECTS TO ADD:
 */

// Sync error tags to localStorage
useEffect(() => {
  localStorage.setItem('mistake_error_tags', JSON.stringify(errorTags));
}, [errorTags]);

// Apply Rule of 3 archiving on improvements change
useEffect(() => {
  const updated = applyRuleOfThree(improvements);
  if (Object.keys(updated).length !== Object.keys(improvements).length) {
    setImprovements(updated);
  }
}, [improvements]);

// Track recent quiz topics for context boost
useEffect(() => {
  const recent = JSON.parse(localStorage.getItem('quiz_history') || '[]')
    .slice(0, 3)
    .map(q => q.Topic);
  setRecentQuizTopics(recent);
  localStorage.setItem('recent_quiz_topics', JSON.stringify(recent));
}, []);

// ───────────────────────────────────────────────────────────────────────────────

/**
 * MEMOIZED CALCULATIONS: Performance optimization
 */

const prioritizedMistakes = useMemo(() => {
  return mistakes
    .map(m => ({
      ...m,
      masteryPriority: calculateMasteryPriority(m),
      masteryState: getMasteryState(m.improvementCount ?? 0)
    }))
    .sort((a, b) => b.masteryPriority - a.masteryPriority);
}, [mistakes, recentQuizTopics]); // Re-calc when quiz history changes

const errorDensityMap = useMemo(() => {
  const map = {};
  mistakes.forEach(m => {
    if (!map[m.Topic]) map[m.Topic] = { errors: 0, total: 0 };
    map[m.Topic].total += m.attemptCount || 1;
    map[m.Topic].errors++;
  });
  return map;
}, [mistakes]);

In [ ]:
// ═══════════════════════════════════════════════════════════════════════════════
// 7. METACOGNITIVE ERROR TAGGING SYSTEM
// ═══════════════════════════════════════════════════════════════════════════════

const ERROR_TYPES = [
  { value: 'misread', label: 'Misread Question', color: 'blue' },
  { value: 'calculation', label: 'Calculation Error', color: 'red' },
  { value: 'conceptual', label: 'Conceptual Gap', color: 'orange' },
  { value: 'careless', label: 'Careless Mistake', color: 'yellow' },
  { value: 'vocab', label: 'Vocabulary Gap', color: 'purple' },
  { value: 'diagram', label: 'Diagram Misread', color: 'pink' },
];

/**
 * ErrorTagSelector: Dropdown to assign error type to a mistake
 */
function ErrorTagSelector({ questionId, currentTag, onTag }) {
  const { t } = useLanguage();
  const [open, setOpen] = useState(false);
  
  return (
    <div className="relative">
      <button
        onClick={() => setOpen(!open)}
        className={`px-3 py-1.5 rounded-lg text-xs font-bold border border-slate-300 flex items-center gap-1 ${
          currentTag
            ? `bg-slate-100 text-slate-700`
            : 'bg-white text-slate-500 hover:border-slate-400'
        }`}
      >
        {currentTag ? ERROR_TYPES.find(e => e.value === currentTag)?.label : t('notebook.tagError')}
        <ChevronDown size={12} />
      </button>
      
      {open && (
        <div className="absolute top-full mt-1 left-0 bg-white border border-slate-200 rounded-lg shadow-lg z-10 min-w-max">
          {ERROR_TYPES.map(type => (
            <button
              key={type.value}
              onClick={() => {
                onTag(questionId, type.value);
                setOpen(false);
              }}
              className={`block w-full text-left px-4 py-2 text-sm hover:bg-slate-50 border-b border-slate-100 last:border-b-0 ${
                currentTag === type.value ? 'bg-blue-50 text-blue-700 font-bold' : ''
              }`}
            >
              <span className={`inline-block w-2 h-2 rounded-full mr-2 bg-${type.color}-500`} />
              {type.label}
            </button>
          ))}
        </div>
      )}
    </div>
  );
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════════════════
// 6. AI DAILY MISSION: Automated Practice Selection with Interleaved Practice
// ═══════════════════════════════════════════════════════════════════════════════

/**
 * selectAIDailyMission: Automatically select 20 questions using ISRS ranking
 * Enforces interleaved practice: ensures questions span ≥3 different topics
 * 
 * @param {Array} mistakes - All mistake questions
 * @returns {Array} Selected 20 questions with topic diversity
 */
function selectAIDailyMission(mistakes) {
  // Calculate priority score for each mistake
  const prioritizedMistakes = mistakes
    .map(m => ({
      ...m,
      masteryPriority: calculateMasteryPriority(m)
    }))
    .sort((a, b) => b.masteryPriority - a.masteryPriority);
  
  // Enforce interleaved practice: collect questions by topic
  const byTopic = {};
  prioritizedMistakes.forEach(m => {
    if (!byTopic[m.Topic]) byTopic[m.Topic] = [];
    byTopic[m.Topic].push(m);
  });
  
  const topicList = Object.keys(byTopic).sort(
    (a, b) => byTopic[b][0].masteryPriority - byTopic[a][0].masteryPriority
  );
  
  // Select questions in round-robin: spread across topics
  const selected = [];
  const topicIndices = {};
  topicList.forEach(t => topicIndices[t] = 0);
  
  while (selected.length < 20 && topicList.some(t => topicIndices[t] < byTopic[t].length)) {
    for (const topic of topicList) {
      if (selected.length >= 20) break;
      if (topicIndices[topic] < byTopic[topic].length) {
        selected.push(byTopic[topic][topicIndices[topic]]);
        topicIndices[topic]++;
      }
    }
  }
  
  // Verify at least 3 different topics
  const topicCount = new Set(selected.map(q => q.Topic)).size;
  if (topicCount < 3) {
    console.warn(`AI Daily Mission: Only ${topicCount} topics, need ≥3 for interleaved practice`);
  }
  
  return selected.slice(0, 20);
}

// ───────────────────────────────────────────────────────────────────────────────

/**
 * Rule of 3: Archive question after 3 consecutive correct answers
 * Move from active mistakes to mistake_archive in localStorage
 */
function applyRuleOfThree(improvements) {
  const archived = JSON.parse(localStorage.getItem('mistake_archive') || '{}');
  const activeImprovements = { ...improvements };
  
  Object.entries(improvements).forEach(([questionId, data]) => {
    if (data.correctCount >= 3) {
      archived[questionId] = { ...data, archivedAt: new Date().toISOString() };
      delete activeImprovements[questionId];
    }
  });
  
  localStorage.setItem('mistake_archive', JSON.stringify(archived));
  localStorage.setItem('mistake_improvements', JSON.stringify(activeImprovements));
  
  return activeImprovements;
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════════════════
// 5. LEARNING ANALYTICS DASHBOARD
// ═══════════════════════════════════════════════════════════════════════════════

/**
 * TopicHeatmap: Error Density (Wrong/Total attempts per topic)
 * Color scale: Yellow (low errors) → Deep Crimson (high errors)
 */
function TopicHeatmap({ mistakes }) {
  const { t } = useLanguage();
  
  const errorDensity = useMemo(() => {
    const topicMap = {};
    
    mistakes.forEach(m => {
      if (!topicMap[m.Topic]) {
        topicMap[m.Topic] = { attempted: 0, wrong: 0 };
      }
      topicMap[m.Topic].wrong++;
      topicMap[m.Topic].attempted += Math.max(m.attemptCount, 1);
    });
    
    // Calculate error density ratio
    return Object.entries(topicMap).map(([topic, data]) => ({
      topic,
      errorDensity: Math.min(1.0, data.wrong / Math.max(data.attempted, 1)),
      wrongCount: data.wrong,
      attemptedCount: data.attempted
    })).sort((a, b) => b.errorDensity - a.errorDensity);
  }, [mistakes]);
  
  const getHeatColor = (density) => {
    if (density < 0.2) return 'bg-yellow-100 text-yellow-800';
    if (density < 0.4) return 'bg-orange-100 text-orange-800';
    if (density < 0.6) return 'bg-red-300 text-red-900';
    if (density < 0.8) return 'bg-red-500 text-red-50';
    return 'bg-crimson-700 text-white';
  };
  
  return (
    <div className="bg-white rounded-xl p-6 border border-slate-200">
      <h3 className="font-bold text-lg text-slate-800 mb-4 flex items-center gap-2">
        <BarChart2 size={20} />
        {t('notebook.errorDensityByTopic')}
      </h3>
      
      <div className="space-y-2">
        {errorDensity.map(({ topic, errorDensity: density, wrongCount, attemptedCount }) => (
          <div key={topic} className="flex items-center gap-4">
            <div className="w-32 font-semibold text-sm text-slate-700 truncate">
              {topic}
            </div>
            
            <div className="flex-1">
              <div className={`h-8 rounded-lg flex items-center justify-center font-bold text-sm transition-all ${getHeatColor(density)}`}>
                {(density * 100).toFixed(0)}% ({wrongCount}/{attemptedCount})
              </div>
            </div>
          </div>
        ))}
      </div>
    </div>
  );
}

// ───────────────────────────────────────────────────────────────────────────────

/**
 * ImprovementTrendChart: Stacked area chart of mastery state progression
 * Shows volume of questions flowing through: Unprocessed → Acquiring → Consolidating → Mastered
 */
function ImprovementTrendChart({ improvements }) {
  const { t } = useLanguage();
  
  const trendData = useMemo(() => {
    const stateHistory = {};
    
    Object.values(improvements).forEach(data => {
      const state = getMasteryState(data.correctCount || 0);
      const dateStr = data.lastCorrect
        ? new Date(data.lastCorrect).toISOString().split('T')[0]
        : new Date().toISOString().split('T')[0];
      
      if (!stateHistory[dateStr]) {
        stateHistory[dateStr] = { Unprocessed: 0, Acquiring: 0, Consolidating: 0, Mastered: 0 };
      }
      stateHistory[dateStr][state.label]++;
    });
    
    return Object.entries(stateHistory)
      .sort((a, b) => new Date(a[0]) - new Date(b[0]))
      .map(([date, states]) => ({ date, ...states }))
      .slice(-14); // Last 14 days
  }, [improvements]);
  
  return (
    <div className="bg-white rounded-xl p-6 border border-slate-200">
      <h3 className="font-bold text-lg text-slate-800 mb-4 flex items-center gap-2">
        <TrendingUp size={20} />
        {t('notebook.improvementTrend')}
      </h3>
      
      <ResponsiveContainer width="100%" height={300}>
        <AreaChart data={trendData}>
          <CartesianGrid strokeDasharray="3 3" />
          <XAxis dataKey="date" />
          <YAxis />
          <Tooltip />
          <Legend />
          <Area type="monotone" dataKey="Unprocessed" stackId="1" stroke="#ef4444" fill="#fecaca" />
          <Area type="monotone" dataKey="Acquiring" stackId="1" stroke="#f59e0b" fill="#fed7aa" />
          <Area type="monotone" dataKey="Consolidating" stackId="1" stroke="#eab308" fill="#fef08a" />
          <Area type="monotone" dataKey="Mastered" stackId="1" stroke="#22c55e" fill="#bbf7d0" />
        </AreaChart>
      </ResponsiveContainer>
    </div>
  );
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════════════════
// 4. CALENDAR HEATMAP: 30-Day Mistake-Clearing Activity Visualization
// ═══════════════════════════════════════════════════════════════════════════════

/**
 * CalendarHeatmap: 7-column grid showing 30 days of mistake-clearing activity
 * Color intensity represents daily activity level
 */
function CalendarHeatmap({ improvements }) {
  const { t } = useLanguage();
  
  // Calculate daily activity data
  const activityMap = useMemo(() => {
    const map = {};
    const now = new Date();
    
    // Initialize last 30 days
    for (let i = 0; i < 30; i++) {
      const d = new Date(now);
      d.setDate(d.getDate() - i);
      const dateStr = d.toISOString().split('T')[0];
      map[dateStr] = 0;
    }
    
    // Count mistakes cleared per day
    Object.values(improvements).forEach(data => {
      if (data.lastCorrect) {
        const dateStr = new Date(data.lastCorrect).toISOString().split('T')[0];
        if (map[dateStr] !== undefined) {
          map[dateStr]++;
        }
      }
    });
    
    return map;
  }, [improvements]);
  
  const days = Object.entries(activityMap).reverse().sort();
  const maxActivity = Math.max(...Object.values(activityMap), 1);
  
  // Color intensity based on activity
  const getColor = (count) => {
    const intensity = count / maxActivity;
    if (intensity === 0) return 'bg-slate-100';
    if (intensity < 0.33) return 'bg-blue-200';
    if (intensity < 0.67) return 'bg-blue-400';
    return 'bg-blue-600';
  };
  
  return (
    <div className="bg-white rounded-xl p-4 border border-slate-200">
      <h3 className="font-bold text-slate-700 mb-4 flex items-center gap-2">
        <Calendar size={16} />
        {t('notebook.mistakeClearingActivity')}
      </h3>
      
      <div className="grid grid-cols-7 gap-1">
        {days.map(([dateStr, count], idx) => {
          const date = new Date(dateStr + 'T00:00:00');
          const weekday = date.toLocaleDateString('en-US', { weekday: 'short' });
          
          return (
            <div key={dateStr} className="flex flex-col items-center">
              {idx < 7 && <span className="text-xs text-slate-400 h-5">{weekday}</span>}
              <div
                className={`w-8 h-8 rounded ${getColor(count)} flex items-center justify-center text-xs font-bold text-slate-700 hover:ring-2 ring-blue-400 transition-all`}
                title={`${dateStr}: ${count} cleared`}
              >
                {count > 0 ? count : ''}
              </div>
            </div>
          );
        })}
      </div>
      
      <div className="flex items-center gap-2 mt-4 text-xs text-slate-500">
        <span>{t('notebook.less')}</span>
        <div className="flex gap-1">
          {[0, 0.33, 0.67, 1.0].map(i => (
            <div key={i} className={`w-3 h-3 rounded ${getColor(i * maxActivity)}`} />
          ))}
        </div>
        <span>{t('notebook.more')}</span>
      </div>
    </div>
  );
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════════════════
// 3. FLOATING UI SMART TOOLTIP: Viewport-Aware with Portal Rendering
// ═══════════════════════════════════════════════════════════════════════════════

/**
 * TooltipWithPortal: Smart tooltip using Floating UI
 * - Automatically detects viewport edges and flips position
 * - Renders to document.body via Portal to avoid z-index/overflow issues
 * - Handles mouse enter/leave for show/hide
 */
function TooltipWithPortal({ trigger, content, placement = 'top' }) {
  const [open, setOpen] = useState(false);
  const arrowRef = useRef(null);
  
  const { refs, floatingStyles, middlewareData } = useFloating({
    placement,
    open,
    onOpenChange: setOpen,
    middleware: [
      offset(10),           // 10px gap from trigger
      flip(),               // Flip to opposite side if too close to edge
      shift({ padding: 8 }) // Shift to keep within viewport
    ],
    whileElementsMounted: autoUpdate  // Update on DOM changes
  });
  
  return (
    <>
      <div
        ref={refs.setReference}
        onMouseEnter={() => setOpen(true)}
        onMouseLeave={() => setOpen(false)}
        className="cursor-help"
      >
        {trigger}
      </div>
      
      {open && createPortal(
        <div
          ref={refs.setFloating}
          style={floatingStyles}
          className="z-[9999] bg-slate-900 text-white text-xs rounded-xl p-3 shadow-2xl ring-1 ring-white/10 max-w-xs pointer-events-none"
        >
          {content}
        </div>,
        document.body
      )}
    </>
  );
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════════════════
// 2. ISRS CALCULATION: Multi-Weighted Mastery Priority Score
// ═══════════════════════════════════════════════════════════════════════════════

/**
 * Calculate comprehensive mastery priority score using weighted integration
 * Score = (U × 0.4) + (D × 0.4) + (R × 0.2)
 * 
 * @param {Object} mistake - Question mistake object
 * @param {number} mistake.lastAttempted - ISO timestamp of last attempt
 * @param {number} mistake.attemptCount - Total attempts on this question
 * @param {string} mistake.Topic - Topic of the question
 * @returns {number} Priority score (higher = more urgent)
 */
function calculateMasteryPriority(mistake) {
  // 1. URGENCY: Ebbinghaus Forgetting Curve
  const now = Date.now();
  const lastAttemptTime = new Date(mistake.lastAttempted).getTime();
  const daysSinceLastAttempt = (now - lastAttemptTime) / (1000 * 60 * 60 * 24);
  
  // U = 2^(days/7) - exponential curve matching forgetting pattern
  const U = Math.pow(2, daysSinceLastAttempt / 7);
  
  // 2. DIFFICULTY: Based on attempt count
  // At 3+ attempts, question is at maximum difficulty (D = 1.0)
  const D = Math.min(1.0, (mistake.attemptCount || 1) / 3);
  
  // 3. RECENCY/CONTEXT: Boost if matches recent quiz topics
  let R = 0.5; // Baseline score
  
  const recentTopics = JSON.parse(localStorage.getItem('recent_quiz_topics') || '[]');
  if (recentTopics.includes(mistake.Topic)) {
    R = 1.5; // 1.5x boost for contextual relevance
  }
  
  // Final weighted score
  const score = (U * 0.4) + (D * 0.4) + (R * 0.2);
  
  return score;
}

// ───────────────────────────────────────────────────────────────────────────────

/**
 * Get current mastery state based on improvement count
 * State machine: 0 (Unprocessed) → 1 (Acquiring) → 2 (Consolidating) → 3 (Mastered)
 */
function getMasteryState(improvementCount = 0) {
  if (improvementCount === 0) return { state: 0, label: 'Unprocessed' };
  if (improvementCount === 1) return { state: 1, label: 'Acquiring' };
  if (improvementCount === 2) return { state: 2, label: 'Consolidating' };
  return { state: 3, label: 'Mastered' };
}

In [ ]:
// ═══════════════════════════════════════════════════════════════════════════════
// 1. SETUP: Dependencies and Imports to Add
// ═══════════════════════════════════════════════════════════════════════════════

// Add to package.json:
// "npm install @floating-ui/react recharts"

// New imports for MistakeNotebookPage.jsx:
import { useFloating, offset, flip, shift, arrow } from '@floating-ui/react';
import { AreaChart, Area, XAxis, YAxis, CartesianGrid, Tooltip, Legend, ResponsiveContainer } from 'recharts';
import { createPortal } from 'react-dom';

// Existing imports to keep:
import React, { useState, useEffect, useMemo, useRef, useCallback } from 'react';
import { useNavigate } from 'react-router-dom';
import { useAuth } from '../contexts/AuthContext';
import { useLanguage } from '../contexts/LanguageContext';

# MistakeNotebookPage Refactoring: Multi-Weighted ISRS System

## Overview
Comprehensive refactoring of MistakeNotebookPage.jsx implementing:
- **Integrated Spaced Repetition System (ISRS)** with weighted mastery scoring
- **Smart Tooltips** using viewport-aware positioning (Floating UI)
- **AI Daily Mission** for automated practice selection with interleaved practice
- **Learning Analytics Dashboard** with heatmaps and trend visualization
- **Calendar Heatmap** for 30-day activity tracking
- **Rule of 3 Archiving** for automatic mastery graduation

### Core Formulas
**MasteryPriority Score**: $\text{Score} = (U \times 0.4) + (D \times 0.4) + (R \times 0.2)$

- **Urgency (U)**: $U = 2^{days/7}$ (Ebbinghaus Forgetting Curve)
- **Difficulty (D)**: Normalized from attempt count (maxed at 1.0 for 3+ attempts)
- **Recency (R)**: Context boost from recent quiz topics (1.5x multiplier if matched)